In [1]:
import h2o
from h2o.automl import H2OAutoML
h2o.init();

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /home/alexander/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp_8my0v2s
  JVM stdout: /tmp/tmp_8my0v2s/h2o_alexander_started_from_python.out
  JVM stderr: /tmp/tmp_8my0v2s/h2o_alexander_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Moscow
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.10
H2O cluster version age:,20 days
H2O cluster name:,H2O_from_python_alexander_9f87sg
H2O cluster total nodes:,1
H2O cluster free memory:,2.585 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [2]:
df = h2o.upload_file("train.csv")
test = h2o.upload_file("test.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
y = "Survived"
df[y] = df[y].asfactor()

In [4]:
aml = H2OAutoML(max_models = 100, max_runtime_secs = 600, seed = 22)
aml.train(y = y, training_frame = df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [5]:
lb = aml.leaderboard
lb.head()

model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_grid_1_AutoML_20191128_085309_model_4,0.873782,0.402652,0.17611,0.353314,0.124831
GBM_grid_1_AutoML_20191128_085309_model_20,0.872858,0.471787,0.179992,0.384612,0.147927
GBM_grid_1_AutoML_20191128_085309_model_10,0.87225,0.507187,0.187039,0.402871,0.162305
XRT_1_AutoML_20191128_085309,0.871824,0.572548,0.186607,0.362879,0.131681
XGBoost_grid_1_AutoML_20191128_085309_model_5,0.871318,0.413699,0.169543,0.355211,0.126175
XGBoost_grid_1_AutoML_20191128_085309_model_10,0.871223,0.411557,0.181694,0.3558,0.126594
GBM_grid_1_AutoML_20191128_085309_model_22,0.870823,0.564367,0.181119,0.433853,0.188228
StackedEnsemble_BestOfFamily_AutoML_20191128_085309,0.870746,0.401357,0.173545,0.351582,0.12361
XGBoost_grid_1_AutoML_20191128_085309_model_6,0.870195,0.41519,0.168057,0.356666,0.127211
XGBoost_3_AutoML_20191128_085309,0.869103,0.410928,0.169327,0.354926,0.125972


In [6]:
pred = aml.predict(test);

xgboost prediction progress: |████████████████████████████████████████████| 100%


/home/alexander/anaconda3/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Cabin' has levels not trained on: [A11, A18, A21, A29, A9, B10, B11, B24, B26, B36, B45, B52 B54 B56, B61, C105, C116, C130, C132, C28, C31, C39, C51, C53, C55 C57, C6, C80, C89, C97, D22, D34, D38, D40, D43, E39 E41, E45, E52, E60, F, F E46, F E57]
  warnings.warn(w)


In [7]:
res = pred.as_data_frame().predict

In [8]:
import pandas as pd

PassengerId = test['PassengerId'].as_data_frame()
pd.concat([PassengerId,res],axis=1,ignore_index = True).to_csv('gender_submission.csv',index=False)